# cute张量抽象层 - 实践

> 用Python模拟理解cute的核心概念

---

本notebook通过Python代码模拟cute的核心概念，帮助你理解：
1. Layout（布局）的概念
2. Shape和Stride如何工作
3. 分块（Tiling）操作
4. 为什么这些抽象对FlashAttention很重要

**注意**：cute是C++模板库，这里用Python模拟其概念，便于理解。


## 1. 环境准备


In [ ]:
import numpy as np
from typing import Tuple, List

print("numpy版本:", np.__version__)
print("\n本notebook模拟cute的概念，不需要GPU。")


## 2. 理解Layout

Layout是cute中最核心的概念。它由Shape和Stride组成，描述了多维数据如何映射到一维内存。


In [ ]:
class Layout:
    """
    模拟cute的Layout概念
    
    Layout = (Shape, Stride)
    - Shape: 每个维度的大小
    - Stride: 每个维度移动一步在线性内存中的偏移
    """
    
    def __init__(self, shape: Tuple[int, ...], stride: Tuple[int, ...] = None):
        self.shape = shape
        # 如果没有提供stride，默认为行优先（C顺序）
        if stride is None:
            stride = self._compute_row_major_stride(shape)
        self.stride = stride
        
    def _compute_row_major_stride(self, shape: Tuple[int, ...]) -> Tuple[int, ...]:
        """计算行优先的stride"""
        stride = []
        s = 1
        for dim in reversed(shape):
            stride.append(s)
            s *= dim
        return tuple(reversed(stride))
    
    def __call__(self, *indices) -> int:
        """计算线性索引"""
        assert len(indices) == len(self.shape)
        offset = 0
        for idx, s in zip(indices, self.stride):
            offset += idx * s
        return offset
    
    def size(self) -> int:
        """总元素数"""
        result = 1
        for s in self.shape:
            result *= s
        return result
    
    def __repr__(self):
        return f"Layout(shape={self.shape}, stride={self.stride})"

# 测试Layout
print("="*60)
print("Layout示例")
print("="*60)

# 行优先的4x3矩阵
row_major = Layout((4, 3), (3, 1))
print(f"\n行优先 4x3 矩阵: {row_major}")
print("索引 (0,0) -> 线性偏移:", row_major(0, 0))
print("索引 (0,2) -> 线性偏移:", row_major(0, 2))
print("索引 (1,0) -> 线性偏移:", row_major(1, 0))
print("索引 (2,1) -> 线性偏移:", row_major(2, 1))

# 列优先的4x3矩阵
col_major = Layout((4, 3), (1, 4))
print(f"\n列优先 4x3 矩阵: {col_major}")
print("索引 (0,0) -> 线性偏移:", col_major(0, 0))
print("索引 (0,2) -> 线性偏移:", col_major(0, 2))
print("索引 (1,0) -> 线性偏移:", col_major(1, 0))
print("索引 (2,1) -> 线性偏移:", col_major(2, 1))


## 3. 可视化行优先 vs 列优先


In [ ]:
def visualize_layout(layout: Layout, name: str):
    """可视化Layout的内存映射"""
    rows, cols = layout.shape
    
    print(f"\n{name}")
    print(f"Shape: {layout.shape}, Stride: {layout.stride}")
    print("-" * 40)
    
    # 逻辑视图
    print("\n逻辑视图 (i, j):")
    for i in range(rows):
        row_str = "  "
        for j in range(cols):
            row_str += f"({i},{j})  "
        print(row_str)
    
    # 线性内存偏移
    print("\n线性内存偏移:")
    for i in range(rows):
        row_str = "  "
        for j in range(cols):
            offset = layout(i, j)
            row_str += f"[{offset:2d}]   "
        print(row_str)
    
    # 内存中的顺序
    max_offset = max(layout(i, j) for i in range(rows) for j in range(cols))
    print(f"\n内存布局 (0 到 {max_offset}):")
    memory = ['--'] * (max_offset + 1)
    for i in range(rows):
        for j in range(cols):
            offset = layout(i, j)
            memory[offset] = f"({i},{j})"
    print("  " + " ".join(memory))

# 可视化
visualize_layout(Layout((3, 4), (4, 1)), "行优先 3x4")
visualize_layout(Layout((3, 4), (1, 3)), "列优先 3x4")


## 4. 模拟cute的Tensor

Tensor = 数据指针 + Layout


In [ ]:
class Tensor:
    """
    模拟cute的Tensor概念
    
    Tensor = Data + Layout
    """
    
    def __init__(self, data: np.ndarray, layout: Layout = None):
        self.data = data.flatten()  # 模拟线性内存
        if layout is None:
            layout = Layout(data.shape)
        self.layout = layout
        
    def __getitem__(self, indices):
        if not isinstance(indices, tuple):
            indices = (indices,)
        offset = self.layout(*indices)
        return self.data[offset]
    
    def __setitem__(self, indices, value):
        if not isinstance(indices, tuple):
            indices = (indices,)
        offset = self.layout(*indices)
        self.data[offset] = value
    
    @property
    def shape(self):
        return self.layout.shape
    
    def __repr__(self):
        return f"Tensor(shape={self.shape}, layout={self.layout})"

# 创建一个Tensor
print("="*60)
print("Tensor示例")
print("="*60)

# 创建3x4矩阵
data = np.arange(12).reshape(3, 4).astype(np.float32)
print("原始numpy数组:")
print(data)

# 包装成cute风格的Tensor
tensor = Tensor(data, Layout((3, 4), (4, 1)))  # 行优先
print(f"\n{tensor}")
print(f"tensor[0, 0] = {tensor[0, 0]}")
print(f"tensor[1, 2] = {tensor[1, 2]}")
print(f"tensor[2, 3] = {tensor[2, 3]}")


## 5. local_tile：获取局部Tile

这是FlashAttention中的核心操作 - 从大矩阵中获取一个小块。


In [ ]:
def local_tile(tensor: Tensor, tile_shape: Tuple[int, int], tile_coord: Tuple[int, int]) -> Tensor:
    """
    模拟cute的local_tile操作
    
    从大Tensor中获取一个tile的视图
    
    参数:
        tensor: 原始Tensor
        tile_shape: tile的形状 (tile_rows, tile_cols)
        tile_coord: tile的坐标 (tile_row_idx, tile_col_idx)
    
    返回:
        指向原始数据的Tensor视图
    """
    tile_rows, tile_cols = tile_shape
    tile_row_idx, tile_col_idx = tile_coord
    
    # 计算tile的起始位置
    start_row = tile_row_idx * tile_rows
    start_col = tile_col_idx * tile_cols
    
    # 计算起始偏移
    start_offset = tensor.layout(start_row, start_col)
    
    # 创建新的Layout（stride保持不变）
    new_layout = Layout(tile_shape, tensor.layout.stride)
    
    # 创建新的Tensor视图（共享底层数据，偏移起始位置）
    class TensorView:
        def __init__(self, data, offset, layout):
            self.data = data
            self.offset = offset
            self.layout = layout
            
        def __getitem__(self, indices):
            if not isinstance(indices, tuple):
                indices = (indices,)
            local_offset = self.layout(*indices)
            return self.data[self.offset + local_offset]
        
        @property
        def shape(self):
            return self.layout.shape
        
        def __repr__(self):
            return f"TensorView(shape={self.shape}, offset={self.offset})"
    
    return TensorView(tensor.data, start_offset, new_layout)

# 演示local_tile
print("="*60)
print("local_tile演示")
print("="*60)

# 创建8x8矩阵
big_data = np.arange(64).reshape(8, 8).astype(np.float32)
big_tensor = Tensor(big_data, Layout((8, 8), (8, 1)))

print("原始8x8矩阵:")
print(big_data)

# 获取不同位置的4x4 tile
print("\n获取4x4的tiles:")
for row_idx in range(2):
    for col_idx in range(2):
        tile = local_tile(big_tensor, (4, 4), (row_idx, col_idx))
        print(f"\nTile[{row_idx}, {col_idx}]: {tile}")
        # 打印tile内容
        for i in range(4):
            row = [f"{tile[i, j]:4.0f}" for j in range(4)]
            print("  " + " ".join(row))


## 6. 模拟线程分区（Partition）

在GPU上，多个线程协作处理一个tile。Partition描述了每个线程负责的数据。


In [ ]:
def partition_for_threads(tensor_shape: Tuple[int, int], 
                          num_threads: int,
                          thread_layout: Tuple[int, int]) -> dict:
    """
    模拟cute的partition操作
    
    将tensor分配给多个线程
    
    参数:
        tensor_shape: tensor的形状
        num_threads: 线程数
        thread_layout: 线程排列方式 (thread_rows, thread_cols)
    
    返回:
        每个线程负责的元素索引
    """
    rows, cols = tensor_shape
    thread_rows, thread_cols = thread_layout
    
    assert thread_rows * thread_cols == num_threads
    
    # 每个线程处理的元素数
    elements_per_thread_row = rows // thread_rows
    elements_per_thread_col = cols // thread_cols
    
    partitions = {}
    
    for thread_id in range(num_threads):
        # 计算线程在网格中的位置
        thread_row = thread_id // thread_cols
        thread_col = thread_id % thread_cols
        
        # 计算该线程负责的元素范围
        start_row = thread_row * elements_per_thread_row
        end_row = start_row + elements_per_thread_row
        start_col = thread_col * elements_per_thread_col
        end_col = start_col + elements_per_thread_col
        
        elements = []
        for i in range(start_row, end_row):
            for j in range(start_col, end_col):
                elements.append((i, j))
        
        partitions[thread_id] = {
            'thread_pos': (thread_row, thread_col),
            'range': ((start_row, end_row), (start_col, end_col)),
            'elements': elements
        }
    
    return partitions

# 演示partition
print("="*60)
print("线程分区演示")
print("="*60)

print("\n场景: 8x8 矩阵，4个线程 (2x2排列)")
partitions = partition_for_threads((8, 8), 4, (2, 2))

for thread_id, info in partitions.items():
    print(f"\nThread {thread_id}:")
    print(f"  位置: {info['thread_pos']}")
    print(f"  行范围: {info['range'][0]}")
    print(f"  列范围: {info['range'][1]}")
    print(f"  元素数: {len(info['elements'])}")

# 可视化分区
print("\n分区可视化 (数字表示thread_id):")
grid = np.zeros((8, 8), dtype=int)
for thread_id, info in partitions.items():
    for (i, j) in info['elements']:
        grid[i, j] = thread_id

for i in range(8):
    print("  " + " ".join(f"{grid[i, j]}" for j in range(8)))


## 7. FlashAttention中的分块示例

让我们模拟FlashAttention如何使用这些概念处理Q、K矩阵。


In [ ]:
def simulate_flash_attention_tiling():
    """
    模拟FlashAttention的分块过程
    """
    # 参数
    seqlen = 16   # 序列长度 (简化)
    head_dim = 8  # 头维度 (简化)
    Br = 4        # Q的块大小
    Bc = 4        # K的块大小
    
    print("="*60)
    print("FlashAttention分块模拟")
    print("="*60)
    print(f"\n参数:")
    print(f"  序列长度 (N): {seqlen}")
    print(f"  头维度 (d): {head_dim}")
    print(f"  Q块大小 (Br): {Br}")
    print(f"  K块大小 (Bc): {Bc}")
    
    # 创建Q和K矩阵
    Q = np.random.randn(seqlen, head_dim).astype(np.float16)
    K = np.random.randn(seqlen, head_dim).astype(np.float16)
    
    print(f"\n矩阵形状:")
    print(f"  Q: {Q.shape}")
    print(f"  K: {K.shape}")
    
    # 计算块数
    num_blocks_q = seqlen // Br
    num_blocks_k = seqlen // Bc
    
    print(f"\n分块信息:")
    print(f"  Q方向块数: {num_blocks_q}")
    print(f"  K方向块数: {num_blocks_k}")
    print(f"  总迭代次数: {num_blocks_q * num_blocks_k}")
    
    # 模拟分块迭代
    print("\n分块迭代过程:")
    print("-" * 40)
    
    for i in range(num_blocks_q):
        # 加载Q的第i块到SRAM
        q_start = i * Br
        q_end = q_start + Br
        Q_block = Q[q_start:q_end, :]  # [Br, d]
        
        print(f"\n外循环 i={i}: Q[{q_start}:{q_end}, :]")
        
        for j in range(num_blocks_k):
            # 加载K的第j块到SRAM
            k_start = j * Bc
            k_end = k_start + Bc
            K_block = K[k_start:k_end, :]  # [Bc, d]
            
            # 计算局部注意力分数
            S_block = Q_block @ K_block.T  # [Br, Bc]
            
            print(f"  内循环 j={j}: K[{k_start}:{k_end}, :] -> S[{q_start}:{q_end}, {k_start}:{k_end}]")
            print(f"    S_block shape: {S_block.shape}")
    
    print("\n" + "-" * 40)
    print("关键优化:")
    print("1. Q块只加载一次，K块在内循环中遍历")
    print("2. S矩阵从不完整存储，只有当前块在SRAM中")
    print("3. 使用online softmax增量计算最终结果")

simulate_flash_attention_tiling()


## 8. cute代码与Python对照

让我们对比cute的C++代码和我们的Python模拟。


In [ ]:
comparison = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                     cute C++ vs Python 模拟对照                               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  1. 创建Layout                                                                ║
║  ──────────────────────────────────────────────────────────────────────────  ║
║  cute C++:                                                                    ║
║    Layout layout = make_layout(make_shape(4, 3), make_stride(3, 1));         ║
║                                                                               ║
║  Python模拟:                                                                  ║
║    layout = Layout((4, 3), (3, 1))                                           ║
║                                                                               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  2. 创建Tensor                                                                ║
║  ──────────────────────────────────────────────────────────────────────────  ║
║  cute C++:                                                                    ║
║    Tensor mQ = make_tensor(make_gmem_ptr(ptr), layout);                      ║
║                                                                               ║
║  Python模拟:                                                                  ║
║    tensor = Tensor(data, layout)                                             ║
║                                                                               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  3. 获取局部Tile                                                              ║
║  ──────────────────────────────────────────────────────────────────────────  ║
║  cute C++:                                                                    ║
║    Tensor tile = local_tile(mQ, tile_shape, tile_coord);                     ║
║                                                                               ║
║  Python模拟:                                                                  ║
║    tile = local_tile(tensor, tile_shape, tile_coord)                         ║
║                                                                               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  4. 分区                                                                      ║
║  ──────────────────────────────────────────────────────────────────────────  ║
║  cute C++:                                                                    ║
║    auto thr_copy = tiled_copy.get_slice(thread_idx);                         ║
║    Tensor tQgQ = thr_copy.partition_S(gQ);                                   ║
║                                                                               ║
║  Python模拟:                                                                  ║
║    partitions = partition_for_threads(shape, num_threads, thread_layout)     ║
║                                                                               ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""
print(comparison)


## 9. 总结

通过这个notebook，我们理解了：

1. **Layout**: 用Shape和Stride描述内存布局
2. **Tensor**: 数据 + Layout的组合
3. **local_tile**: 从大矩阵中获取小块视图
4. **Partition**: 将数据分配给多个线程

这些抽象让FlashAttention的实现变得更加清晰和可维护。

---

## 📚 延伸阅读

- [cute Layout教程](https://github.com/NVIDIA/cutlass/blob/main/media/docs/cute/01_layout.md)
- [cute Tensor教程](https://github.com/NVIDIA/cutlass/blob/main/media/docs/cute/02_tensor.md)
- [cute MMA教程](https://github.com/NVIDIA/cutlass/blob/main/media/docs/cute/0t_mma_atom.md)
